# pattern for simple amendments to annex

In [1]:
#table annex with numbers
import pandas
import os
import requests
import re
from bs4 import BeautifulSoup

url = 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32022R0340'
#url = 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32011D0236'
response = requests.get(url)
print(response.status_code)
data = response.text
data

200


'<?xml version="1.0" encoding="UTF-8"?>\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML//EN" "xhtml-strict.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<!-- CONVEX # converter_version:9.15.0 # generated_on:20240304-2034 # ELI version:0.10 -->\r\n<head><meta name="format-detection" content="telephone=no"/><meta http-equiv=\'X-UA-Compatible\' content=\'IE=edge\'/><meta name=\'viewport\' content=\'width=device-width, initial-scale=1\'><meta name=\'WT.z_usr_lan\' content=\'EN\'>\n   <meta http-equiv="content-type" content="text/html; charset=utf-8"/>\n   <script type="text/javascript" src="/ruxitagentjs_ICANVfgqrux_10289240325103055.js" data-dtconfig="app=47d4c64c3b67ec69|agentId=daf5c5adbd7ceee|featureHash=ICANVfgqrux|rdnt=1|uxrgce=1|bp=3|cuc=m097nmfl|mel=100000|mb=null|dpvc=1|iub=null|ssv=4|lastModification=1721139136808|tp=500,50,0|agentUri=/ruxitagentjs_ICANVfgqrux_10289240325103055.js|reportUrl=/rb_39a3e95b-5423-482c-879b-99ef235dffeb|rid=RID_1150611987|rpid=254571597|domain=eu

In [2]:
#get the document information
url= re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data)
publication_date = re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
title = re.findall(r'class="oj-doc-ti"[^>]*>([\s\S]*?)<\/p>', data)
celex= re.findall(r'uri=CELEX%([^"]+)', data)
regulation_match = re.findall(r'(COUNCIL IMPLEMENTING.*?)</p', data)
legal_decision = re.findall(r'<p[^>]*>(.*?implementing.*?)</p>',data)
legal_base = re.findall(r'<p[^>]*>(.*?regard to.*?)</p>',data)
print(url),print(publication_date),print(title),print(celex),print(regulation_match),print(legal_decision),print(legal_base)

['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32011D0236']
['14.4.2011']
['\n            COUNCIL IMPLEMENTING DECISION 2011/236/CFSP', 'of 12 April 2011\n         ', 'implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya', 'ANNEX I', 'ANNEX II', 'ANNEX III', 'ANNEX IV']
['3A32011D0236']
['COUNCIL IMPLEMENTING DECISION 2011/236/CFSP']
['implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya']
['Having regard to Council Decision 2011/137/CFSP of 28 February 2011 concerning restrictive measures in view of the situation in Libya\xa0<a id="ntc1-L_2011100EN.01005801-E0001" href="#ntr1-L_2011100EN.01005801-E0001">(<span class="oj-super oj-note-tag">1</span>)</a>, and in particular Article 8(1) and (2) thereof, in conjunction with Article 31(2) of the Treaty on European Union,']


(None, None, None, None, None, None, None)

In [3]:
#search for UN to add the UN dummy indicator
UN_pattern = re.search(r'UN',data)

UN = None
for i in data:
    if UN_pattern == None:
        UN = 0
    else:
        UN = 1

In [10]:
#get information on the individuals listed
listing_position = re.findall(r'class="oj-bold">(\d+.*?)',data)
print(listing_position)
name = re.findall(r'<span class="oj-bold">\d+\.\s+(.*?)</span>',data)
#\s+\(alias:\s+(.*?)\)\.)
print(name)

[]
[]


In [5]:
Identifying_Information = re.findall(r'<span class="oj-bold">Other information</span>:</p>\s*<p class="oj-normal">(.*?)</p>',data,re.DOTALL)
Identifying_Information

[]

In [6]:
date_pattern = re.findall(r'<span class="oj-bold">Date of UN designation</span>:\s*(.*?)\n',data)
date_pattern

[]

In [7]:
# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# List to store the parsed data
sanction_list = []

sanction_list.append({
        'URL': url,
        'CELEX':celex,
        'law_number' :title[0],
        'Publication Date': publication_date,
        'legal_base' : title[2],
        'Name': name,
        'Position_list': listing_position,
        'DOB': re.findall(r'>Date of Birth</span>(.*?)</p>',data),
        'POB': re.findall(r'Place of Birth</span>(.*?)</p>',data),
        'adress':  re.findall(r'Adress</span>(.*?)</p>',data),
        'nationality':  re.findall(r'Nationality:</span>(.*?)</p>',data),
        'Identifying Information':Identifying_Information,
        'Reason':  re.findall(r'Designation</span>(.*?)</p>',data),
        'UN_sanction_date' :  date_pattern,
        'Date of Listing': publication_date,
        'UN_sanctions_dummy' : UN
    })

# Display the results
for person in sanction_list:
    print(person)


{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32011D0236'], 'CELEX': ['3A32011D0236'], 'law_number': '\n            COUNCIL IMPLEMENTING DECISION 2011/236/CFSP', 'Publication Date': ['14.4.2011'], 'legal_base': 'implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya', 'Name': [], 'Position_list': [], 'DOB': [], 'POB': [], 'adress': [], 'nationality': [], 'Identifying Information': [], 'Reason': [], 'UN_sanction_date': [], 'Date of Listing': ['14.4.2011'], 'UN_sanctions_dummy': 1}


In [8]:
sanction_list

[{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32011D0236'],
  'CELEX': ['3A32011D0236'],
  'law_number': '\n            COUNCIL IMPLEMENTING DECISION 2011/236/CFSP',
  'Publication Date': ['14.4.2011'],
  'legal_base': 'implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya',
  'Name': [],
  'Position_list': [],
  'DOB': [],
  'POB': [],
  'adress': [],
  'nationality': [],
  'Identifying Information': [],
  'Reason': [],
  'UN_sanction_date': [],
  'Date of Listing': ['14.4.2011'],
  'UN_sanctions_dummy': 1}]